In [ ]:
pip install dash

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 68.9 MB/s eta 0:00:00


In [ ]:
 !pip install -U kaleido

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from dash import Dash, html, dcc

# read in COVID-19 data
url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
df = pd.read_csv(url)

# preprocess data
df = df.groupby('Country/Region').sum().reset_index()
df = df.melt(id_vars=['Country/Region', 'Lat', 'Long'], var_name='Date', value_name='Deaths')
df['Date'] = pd.to_datetime(df['Date'])
df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')
df['Confirmed'] = 0

# create interactive map
fig_map = px.scatter_geo(df, lat='Lat', lon='Long', color='Deaths',
                         hover_name='Country/Region', hover_data=['Deaths'],
                         animation_frame='Date', projection='natural earth')
fig_map.update_layout(title='COVID-19 Deaths by Country')

# create line chart of daily cases and deaths
df['Daily Cases'] = df.groupby('Country/Region')['Confirmed'].diff()
df['Daily Deaths'] = df.groupby('Country/Region')['Deaths'].diff()
fig_line = px.line(df, x='Date', y=['Daily Cases', 'Daily Deaths'], color_discrete_sequence=['blue', 'red'])
fig_line.update_layout(title='Daily New Cases and Deaths by Country')

# create dropdown menu to select countries for bar chart
options = [{'label': i, 'value': i} for i in df['Country/Region'].unique()]
dropdown = dcc.Dropdown(id='dropdown', options=options, value='US')

# create bar chart of total cases and deaths by country
def create_bar_chart(country):
    df_country = df[df['Country/Region'] == country]
    df_country = df_country.tail(1)[['Country/Region', 'Confirmed', 'Deaths']]
    fig_bar = go.Figure(data=[go.Bar(x=['Confirmed', 'Deaths'], y=[df_country['Confirmed'].item(), df_country['Deaths'].item()], 
                                     text=[f"{df_country['Confirmed'].item():,}", f"{df_country['Deaths'].item():,}"],
                                     textposition='auto', marker=dict(color=['#1f77b4', '#d62728']))])
    fig_bar.update_layout(title=f'Total Cases and Deaths in {country}')
    return fig_bar

# define app layout
app = Dash(__name__)
app.layout = html.Div([
    html.H1('COVID-19 Dashboard', style={'text-align': 'center'}),
    html.Div([
        dcc.Graph(figure=fig_map, id='map', style={'width': '50%', 'display': 'inline-block'}),
        dcc.Graph(figure=fig_line, id='line', style={'width': '50%', 'display': 'inline-block'})
    ]),
    html.Div([
        html.Div([dropdown], style={'width': '30%', 'display': 'inline-block'}),
        html.Div(id='bar-chart', style={'width': '70%', 'display': 'inline-block'})
    ])
])

# define app callback for bar chart
@app.callback(
    dash.dependencies.Output('bar-chart', 'figure'),
    [dash.dependencies.Input('dropdown', 'value')]
)
def update_bar_chart(country):
    fig_bar = create_bar_chart(country)
    return fig_bar

if __name__ == '__main__':
  app.run_server(debug=True, port=8051)

Dash is running on http://127.0.0.1:8051/



INFO:dash.dash:Dash is running on http://127.0.0.1:8051/



 * Serving Flask app '__main__'
 * Debug mode: on
